In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")
NESSIE_URI = os.getenv("NESSIE_URI")

MASTER = "spark://spark-master:7077"

jar_packages = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1",
    "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.99.0",
    #"software.amazon.awssdk:bundle:2.28.13",
    #"software.amazon.awssdk:url-connection-client:2.28.13",
    "org.apache.iceberg:iceberg-aws-bundle:1.6.1"
  ]

spark_extensions = [
    "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "org.projectnessie.spark.extensions.NessieSparkSessionExtensions"
]
conf = (
    pyspark.SparkConf()
    .setAppName("Silver Iceberg")
    .set("spark.master", MASTER)
    .set("spark.jars.packages", ','.join(jar_packages))
    .set("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
    .set("spark.executor.memory", "2g")
    .set("spark.executor.cores", "2")
    .set("spark.executor.instances", "1")
    .set("spark.driver.memory", "2g")
    .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .set("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .set("spark.sql.catalog.nessie.s3.endpoint", S3_ENDPOINT)
    .set("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .set("spark.sql.catalog.nessie.uri", NESSIE_URI)
    .set("spark.sql.catalog.nessie.ref", "main")
    .set("spark.sql.catalog.nessie.authentication.type", "NONE")
    .set("spark.sql.catalog.nessie.cache-enabled", "false")
    .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
    .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
    .set("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT)
    .set("spark.hadoop.fs.s3a.path.style.access", "true")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
print(spark)

:: loading settings :: url = jar:file:/opt/bitnami/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-123893f1-6319-41c3-907f-df95f78551af;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.99.0 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.6.1 in central
:: resolution report :: resolve 2537ms :: artifacts dl 90ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.6.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.99.0 from ce

In [3]:
spark

In [2]:
from pyspark.sql import Window
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, isnan, when, count, row_number


In [4]:
tabelas = ["enem_2019", "enem_2020", "enem_2021", "enem_2022", "enem_2023"]

# Variável para armazenar o DataFrame final
tabela_enem_silver = None

# Loop para ler cada tabela e realizar a união
for tabela in tabelas:
    table_temp = spark.read.format("iceberg").load(f"nessie.bronze.{tabela}")

    if tabela_enem_silver is None:
        # Atribui a primeira tabela
        tabela_enem_silver = table_temp 
    else:
        # Faz a união com o tabela existente
        tabela_enem_silver = tabela_enem_silver.union(table_temp)

window_spec = Window.orderBy("NU_INSCRICAO")

# Adicionando a coluna 'ID' com o número da linha
tabela_enem_silver = tabela_enem_silver.withColumn("ID", row_number().over(window_spec))

# Mostrar o DataFrame resultante
tabela_enem_silver.show()

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+---------+------------+----------------+--------------------+---------+---------+----------------------+------------------+---------------+------------------+--------------------+-----------+-----------+--------------+--------------+--------------+--------------+-----------+-----------+-----------+-----------+----------+----------+----------+----------+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+-----------------+-------------+-------------+-------------+-------------+-------------+---------------+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCL

In [5]:
tabela_enem_silver.select("ID", "NU_INSCRICAO", "NU_ANO", "TP_FAIXA_ETARIA", "TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA").show(10)

+---+------------+------+---------------+-------+---------------+-----------+
| ID|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|
+---+------------+------+---------------+-------+---------------+-----------+
|  1|190001004627|  2019|              6|      M|              1|          3|
|  2|190001004628|  2019|              1|      F|              1|          3|
|  3|190001004629|  2019|              3|      F|              1|          1|
|  4|190001004630|  2019|              8|      M|              0|          3|
|  5|190001004631|  2019|              8|      M|              1|          3|
|  6|190001004632|  2019|             12|      M|              1|          1|
|  7|190001004633|  2019|             11|      F|              2|          3|
|  8|190001004634|  2019|             11|      F|              0|          3|
|  9|190001004635|  2019|              4|      M|              1|          3|
| 10|190001004636|  2019|              2|      F|              1

In [7]:
tabela_enem_silver.select('NU_ANO').groupBy('NU_ANO').count().orderBy(F.col('NU_ANO'), ascending=True).show()

+------+-------+
|NU_ANO|  count|
+------+-------+
|  2019|5095171|
|  2020|5783109|
|  2021|3389832|
|  2022|3476105|
|  2023|3933955|
+------+-------+



In [8]:
tabela_enem_silver.printSchema()

root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: integer (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: string (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- TP_ST_CONCLUSAO: integer (nullable = true)
 |-- TP_ANO_CONCLUIU: integer (nullable = true)
 |-- TP_ESCOLA: integer (nullable = true)
 |-- TP_ENSINO: integer (nullable = true)
 |-- IN_TREINEIRO: integer (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: integer (nullable = true)
 |-- TP_LOCALIZACAO_ESC: integer (nullable = true)
 |-- TP_SIT_FUNC_ESC: integer (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer

In [9]:
# Lista de colunas que precisam ser convertidas
colunas_para_utilizar_o_cast = [
    "TP_FAIXA_ETARIA", "TP_SEXO", "TP_ESTADO_CIVIL", "TP_COR_RACA",
    "TP_NACIONALIDADE", "TP_ST_CONCLUSAO", "TP_ANO_CONCLUIU",
    "TP_ESCOLA", "TP_ENSINO", "IN_TREINEIRO", "TP_DEPENDENCIA_ADM_ESC",
    "TP_LOCALIZACAO_ESC", "TP_SIT_FUNC_ESC", "TP_PRESENCA_CN",
    "TP_PRESENCA_CH", "TP_PRESENCA_LC", "TP_PRESENCA_MT", "CO_PROVA_CN",
    "CO_PROVA_CH", "CO_PROVA_LC", "CO_PROVA_MT", "TP_LINGUA", "TP_STATUS_REDACAO"
]

# Aplicar o cast apenas nas colunas especificadas
for col in colunas_para_utilizar_o_cast:
    tabela_enem_silver = tabela_enem_silver.withColumn(col, tabela_enem_silver[col].cast(StringType()))

# Verificar o esquema atualizado
tabela_enem_silver.printSchema()


root
 |-- NU_INSCRICAO: string (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- TP_FAIXA_ETARIA: string (nullable = true)
 |-- TP_SEXO: string (nullable = true)
 |-- TP_ESTADO_CIVIL: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_NACIONALIDADE: string (nullable = true)
 |-- TP_ST_CONCLUSAO: string (nullable = true)
 |-- TP_ANO_CONCLUIU: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- TP_ENSINO: string (nullable = true)
 |-- IN_TREINEIRO: string (nullable = true)
 |-- CO_MUNICIPIO_ESC: integer (nullable = true)
 |-- NO_MUNICIPIO_ESC: string (nullable = true)
 |-- CO_UF_ESC: integer (nullable = true)
 |-- SG_UF_ESC: string (nullable = true)
 |-- TP_DEPENDENCIA_ADM_ESC: string (nullable = true)
 |-- TP_LOCALIZACAO_ESC: string (nullable = true)
 |-- TP_SIT_FUNC_ESC: string (nullable = true)
 |-- CO_MUNICIPIO_PROVA: integer (nullable = true)
 |-- NO_MUNICIPIO_PROVA: string (nullable = true)
 |-- CO_UF_PROVA: integer (nullable 

In [13]:
from pyspark.sql.functions import col, isnan, when, count

def nulos_por_coluna(df):
    """
    Função para iterar sobre todas as colunas e contar valores nulos.

    Args:
    - df (DataFrame): o Dataframe.

    Prints:
    - Nome da coluna e a quantidade de valores nulos.
    """
    # iteração sobre as colunas
    for column in df.columns:
        # Contar valores nulos
        null_count = df.select(
            count(when(col(column).isNull() | isnan(column), 1)).alias("null_count")
        ).collect()[0]["null_count"]
        
        # Printar o resultado
        print(f"Coluna: {column}, Valores nulos: {null_count}")

# Chamar a função passando o DataFrame
nulos_por_coluna(tabela_enem_silver)


Coluna: NU_INSCRICAO, Valores nulos: 0


Coluna: NU_ANO, Valores nulos: 0


Coluna: TP_FAIXA_ETARIA, Valores nulos: 0


Coluna: TP_SEXO, Valores nulos: 0


Coluna: TP_ESTADO_CIVIL, Valores nulos: 0


...

Coluna: Q022, Valores nulos: 95968


Coluna: Q023, Valores nulos: 95968


Coluna: Q024, Valores nulos: 95968


Coluna: Q025, Valores nulos: 95968


Coluna: ID, Valores nulos: 0


In [15]:
""" 
    colunas numericas será substituido por '0'
    colunas string será substituido por 'Desconhecido'
"""
tabela_enem_silver = tabela_enem_silver.fillna(0, subset=["CO_MUNICIPIO_ESC", "CO_UF_ESC", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_COMP1",
                                              "NU_NOTA_COMP2", "NU_NOTA_COMP3", "NU_NOTA_COMP4", "NU_NOTA_COMP5", "NU_NOTA_REDACAO"])
                                   
tabela_enem_silver = tabela_enem_silver.fillna("Desconhecido", subset=[
    "TP_DEPENDENCIA_ADM_ESC", "TP_ENSINO", "TP_LOCALIZACAO_ESC", "TP_SIT_FUNC_ESC", 
    "NO_MUNICIPIO_ESC", "SG_UF_ESC", "CO_PROVA_CN", "CO_PROVA_CH", "CO_PROVA_LC", 
    "CO_PROVA_MT", "TX_RESPOSTAS_CN", "TX_RESPOSTAS_CH", "TX_RESPOSTAS_LC", 
    "TX_RESPOSTAS_MT", "TX_GABARITO_CN", "TX_GABARITO_CH", "TX_GABARITO_LC", 
    "TX_GABARITO_MT", "TP_STATUS_REDACAO", "Q001", "Q002", "Q003", "Q004", "Q005", 
    "Q006", "Q007", "Q008", "Q009", "Q010", "Q011", "Q012", "Q013", "Q014", "Q015", 
    "Q016", "Q017", "Q018", "Q019", "Q020", "Q021", "Q022", "Q023", "Q024", "Q025"
])

In [16]:
verificar_nulos = [
    "CO_MUNICIPIO_ESC", "CO_UF_ESC", "NU_NOTA_CN", "NU_NOTA_CH", "NU_NOTA_LC", "NU_NOTA_MT", "NU_NOTA_COMP1",
      "NU_NOTA_COMP2", "NU_NOTA_COMP3", "NU_NOTA_COMP4", "NU_NOTA_COMP5", "NU_NOTA_REDACAO", "TP_DEPENDENCIA_ADM_ESC", "TP_ENSINO", "TP_LOCALIZACAO_ESC", "TP_SIT_FUNC_ESC", 
    "NO_MUNICIPIO_ESC", "SG_UF_ESC", "CO_PROVA_CN", "CO_PROVA_CH", "CO_PROVA_LC", 
    "CO_PROVA_MT", "TX_RESPOSTAS_CN", "TX_RESPOSTAS_CH", "TX_RESPOSTAS_LC", 
    "TX_RESPOSTAS_MT", "TX_GABARITO_CN", "TX_GABARITO_CH", "TX_GABARITO_LC", 
    "TX_GABARITO_MT", "TP_STATUS_REDACAO", "Q001", "Q002", "Q003", "Q004", "Q005", 
    "Q006", "Q007", "Q008", "Q009", "Q010", "Q011", "Q012", "Q013", "Q014", "Q015", 
    "Q016", "Q017", "Q018", "Q019", "Q020", "Q021", "Q022", "Q023", "Q024", "Q025"]

In [18]:
def nulos_por_coluna(df, verificar_nulos):
    """
    Função para iterar sobre todas as colunas e contar valores nulos.

    Args:
    - df (DataFrame): o Dataframe.
    - verificar nulos (list): Lista de nomes que irá vericar os null values
    Prints:
    - Nome da coluna e a quantidade de valores nulos.
    """
    # iteração sobre as colunas
    for column in verificar_nulos:
        # Contar valores nulos
        null_count = df.select(
            count(when(col(column).isNull() | isnan(column), 1)).alias("null_count")
        ).collect()[0]["null_count"]
        
        # Printar o resultado
        print(f"Coluna: {column}, Valores nulos: {null_count}")

# Chamar a função passando o DataFrame
nulos_por_coluna(tabela_enem_silver, verificar_nulos)


Coluna: CO_MUNICIPIO_ESC, Valores nulos: 0


Coluna: CO_UF_ESC, Valores nulos: 0


Coluna: NU_NOTA_CN, Valores nulos: 0


Coluna: NU_NOTA_CH, Valores nulos: 0


Coluna: NU_NOTA_LC, Valores nulos: 0


...

Coluna: Q021, Valores nulos: 0


Coluna: Q022, Valores nulos: 0


Coluna: Q023, Valores nulos: 0


Coluna: Q024, Valores nulos: 0


Coluna: Q025, Valores nulos: 0


In [19]:
spark.sql("""
CREATE NAMESPACE IF NOT EXISTS nessie.silver
""")

DataFrame[]

In [20]:
spark.sql("""
CREATE OR REPLACE TABLE nessie.silver.tabela_enem_silver (
    NU_INSCRICAO STRING,
    NU_ANO INT,
    TP_FAIXA_ETARIA STRING,
    TP_SEXO STRING,
    TP_ESTADO_CIVIL STRING,
    TP_COR_RACA STRING,
    TP_NACIONALIDADE STRING,
    TP_ST_CONCLUSAO STRING,
    TP_ANO_CONCLUIU STRING,
    TP_ESCOLA STRING,
    TP_ENSINO STRING,
    IN_TREINEIRO STRING,
    CO_MUNICIPIO_ESC INT,
    NO_MUNICIPIO_ESC STRING,
    CO_UF_ESC INT,
    SG_UF_ESC STRING,
    TP_DEPENDENCIA_ADM_ESC STRING,
    TP_LOCALIZACAO_ESC STRING,
    TP_SIT_FUNC_ESC STRING,
    CO_MUNICIPIO_PROVA INT,
    NO_MUNICIPIO_PROVA STRING,
    CO_UF_PROVA INT,
    SG_UF_PROVA STRING,
    TP_PRESENCA_CN STRING,
    TP_PRESENCA_CH STRING,
    TP_PRESENCA_LC STRING,
    TP_PRESENCA_MT STRING,
    CO_PROVA_CN STRING,
    CO_PROVA_CH STRING,
    CO_PROVA_LC STRING,
    CO_PROVA_MT STRING,
    NU_NOTA_CN DOUBLE,
    NU_NOTA_CH DOUBLE,
    NU_NOTA_LC DOUBLE,
    NU_NOTA_MT DOUBLE,
    TX_RESPOSTAS_CN STRING,
    TX_RESPOSTAS_CH STRING,
    TX_RESPOSTAS_LC STRING,
    TX_RESPOSTAS_MT STRING,
    TP_LINGUA STRING,
    TX_GABARITO_CN STRING,
    TX_GABARITO_CH STRING,
    TX_GABARITO_LC STRING,
    TX_GABARITO_MT STRING,
    TP_STATUS_REDACAO STRING,
    NU_NOTA_COMP1 DOUBLE,
    NU_NOTA_COMP2 DOUBLE,
    NU_NOTA_COMP3 DOUBLE,
    NU_NOTA_COMP4 DOUBLE,
    NU_NOTA_COMP5 DOUBLE,
    NU_NOTA_REDACAO DOUBLE,
    Q001 STRING,
    Q002 STRING,
    Q003 STRING,
    Q004 STRING,
    Q005 STRING,
    Q006 STRING,
    Q007 STRING,
    Q008 STRING,
    Q009 STRING,
    Q010 STRING,
    Q011 STRING,
    Q012 STRING,
    Q013 STRING,
    Q014 STRING,
    Q015 STRING,
    Q016 STRING,
    Q017 STRING,
    Q018 STRING,
    Q019 STRING,
    Q020 STRING,
    Q021 STRING,
    Q022 STRING,
    Q023 STRING,
    Q024 STRING,
    Q025 STRING,
    ID INT
)
USING iceberg
PARTITIONED BY (NU_ANO)
LOCATION 's3a://silver/tabela_enem_silver/'
""")

DataFrame[]

In [21]:
tabela_enem_silver.write.format("iceberg").mode("overwrite").saveAsTable("nessie.silver.tabela_enem_silver")

In [22]:
spark.sql(""" SELECT COUNT(*) FROM nessie.silver.tabela_enem_silver""").show()

+--------+
|count(1)|
+--------+
|21678172|
+--------+



In [23]:
spark.sql(""" SELECT
    ID
    NU_INSCRICAO,
    NU_ANO,
    TP_FAIXA_ETARIA,
    TP_SEXO,
    TP_ESTADO_CIVIL,
    TP_COR_RACA,
    TP_NACIONALIDADE,
    TP_ST_CONCLUSAO,
    TP_ANO_CONCLUIU,
    TP_ESCOLA,
    TP_ENSINO,
    IN_TREINEIRO
    FROM nessie.silver.tabela_enem_silver""").show(20)

+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+------------+------------+
|NU_INSCRICAO|NU_ANO|TP_FAIXA_ETARIA|TP_SEXO|TP_ESTADO_CIVIL|TP_COR_RACA|TP_NACIONALIDADE|TP_ST_CONCLUSAO|TP_ANO_CONCLUIU|TP_ESCOLA|   TP_ENSINO|IN_TREINEIRO|
+------------+------+---------------+-------+---------------+-----------+----------------+---------------+---------------+---------+------------+------------+
|     4583466|  2019|             11|      F|              1|          1|               1|              1|             11|        1|           1|           0|
|     4583467|  2019|              9|      F|              1|          2|               1|              1|              6|        1|           1|           0|
|     4583468|  2019|              7|      F|              1|          3|               1|              1|              5|        1|Desconhecido|           0|
|     4583469|  2019|             14|      F| 

In [25]:
spark.stop()